In this exercise, you'll build a project management assistant using OpenAI API Function Calling
A .csv file is used to simulate reading and writing from a database or project management tool API.  Follow the directions in the starter code below, and try to build the functions and function calling logic before you look at the solution on the next page! 

### Overview

* Essential libraries are imported and the OpenAI API key is set.
* Functions are created to read and write data from a CSV file using Pandas dataframes. These functions align with the assistant's capabilities, namely
    * task retrieval and status updates
    * project reporting and analytics
    * resource allocation and scheduling.
    * Each function returns a JSON string as output.
* The conversation with the assistant is structured through messages, including a system message defining the assistant's roles and a user message for input. The user prompts are designed to trigger specific functions.
* Define functions to OpenAI. Each function is named and described with parameters that align with the Python functions.
* After sending the prompts and tool definitions to the model, the response is received and parsed to identify the function calls. The function identified in the response is then executed.
* The notebook runs the identified function with the necessary arguments and appends the resulting JSON output to the messages dictionary. This output is then used in a second call to the model, which augments the data from the function and generates a natural language response.

Import necessary libraries

In [1]:
import os
import json
import pandas as pd
from openai import OpenAI



client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.environ["OPENAI_API_KEY"]
)

First, define the Python functions that will read and write from the project_management.csv file using Pandas dataframes. This code uses Pandas dataframes to read and write from the .csv file. 

We define 3 tasks our project management assistant can perform. 

Each function returns a JSON string as output

In [2]:
# Load the project management data
df_original = pd.read_csv('project_management_original.csv')
display(df_original)

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,Jane Doe,In Progress,High,2023-08-01,2023-07-01,2023-07-10,10,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,completed,Medium,2023-08-15,2023-07-01,2023-12-15,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


### Utility Functions

In [3]:
def display_and_delete_file(file_path):

    # Check if the file exists before attempting to display and delete
    if os.path.exists(file_path):
        # display the content
        df = pd.read_csv(file_path)
        display(df)
        # delete the file
        os.remove(file_path)
        print(f"File '{file_path}' has been deleted.")
    else:
        print(f"File '{file_path}' does not exist.")

### Project Management Functions

In [4]:
def task_retrieval_and_status_updates(task_id, status, last_updated):
    """Retrieve and update task status"""

    df = df_original.copy()  # edit from orginal copy (for code testing)

    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.loc[df['Task ID'] == task_id, 'Last Updated'] = last_updated
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    return json.dumps(task.to_dict())


def project_reporting_and_analytics(project_id):
    """Generate reports on project progress and team performance"""

    df = df_original.copy()  # edit from orginal copy (for code testing)

    project = df.loc[df['Project ID'] == project_id]
    return json.dumps(project.to_dict())


def resource_allocation_and_scheduling(task_id, assigned_to, time_estimate, due_date, status):
    """Allocate tasks based on current workloads and schedules"""

    df = df_original.copy()  # edit from orginal copy (for code testing)

    df.loc[df['Task ID'] == task_id, 'Assigned To'] = assigned_to
    df.loc[df['Task ID'] == task_id, 'Time Estimate'] = time_estimate
    df.loc[df['Task ID'] == task_id, 'Due Date'] = due_date
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    return json.dumps(task.to_dict())

Next, we'll build the project management assistant conversation. 

We'll define the messages to send to the model, including a tools dictionary that defines a list of tools, which are the functions that are available to the model to identify and parse parameters for.

In [5]:
system_prompt = """You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. 
You can provide updates on projects, assign tasks to team members, and schedule meetings. 
You understand project management terminology and are capable of parsing detailed project data. 
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."""


tool_task_retrieval_and_status_updates = {
    "type": "function",
    "function": {
        "name": "task_retrieval_and_status_updates",
        "description": "Retrieve and update task status",
        "parameters": {
            "type": "object",
            "properties": {
                "task_id": {
                    "type": "integer",
                    "description": "The unique identifier for the task"
                },
                "status": {
                    "type": "string",
                    "description": "The new status of the task"
                },
                "last_updated": {
                    "type": "string",
                    "description": "The date of the last status update or change to the task"
                }
            },
            "required": ["task_id", "status", "last_updated"]
        }
    }
}

tool_project_reporting_and_analytics = {
    "type": "function",
    "function": {
        "name": "project_reporting_and_analytics",
        "description": "Generate reports on project progress and team performance",
        "parameters": {
            "type": "object",
            "properties": {
                "project_id": {
                    "type": "integer",
                    "description": "The unique identifier for the project"
                }
            }
        }, 
        "required": ["project_id"]
    }
}

tool_resource_allocation_and_scheduling = {
    "type": "function",
    "function": {
        "name": "resource_allocation_and_scheduling",
        "description": "Allocate tasks based on current workloads and schedules",
        "parameters": {
            "type": "object",
            "properties": {
                "task_id": {
                    "type": "integer",
                    "description": "The unique identifier for the task"
                },
                "assigned_to": {
                    "type": "string",
                    "description": "The user ID or name of the person to whom the task is assigned"
                },
                "time_estimate": {
                    "type": "integer",
                    "description": "An estimate of the time required to complete the task"
                },
                "due_date": {
                    "type": "string",
                    "description": "The deadline for the task completion"
                },
                "status": {
                    "type": "string",
                    "description": "The current status of the task"
                }
            }
        },
        "required": ["task_id", "assigned_to", "time_estimate", "due_date", "status"]
    }
}

In [6]:
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall


def execute_function(tool_call: ChatCompletionMessageToolCall):

    # list the available functions and their corresponding python functions
    available_functions = {
        "task_retrieval_and_status_updates": task_retrieval_and_status_updates,
        "project_reporting_and_analytics": project_reporting_and_analytics,
        "resource_allocation_and_scheduling": resource_allocation_and_scheduling,
    }  

    function_name = tool_call.function.name  # get the name of the function to call
    function_to_call = available_functions[function_name]
    function_args: dict = json.loads(tool_call.function.arguments) 

    if function_name == 'task_retrieval_and_status_updates': # call the Python function
        function_response = function_to_call(
            task_id=function_args.get("task_id"),
            status=function_args.get("status"),
            last_updated=function_args.get("last_updated")
        )
    elif function_name == 'project_reporting_and_analytics':
        function_response = function_to_call(
            project_id=function_args.get("project_id")
        )
    elif function_name == 'resource_allocation_and_scheduling':
        function_response = function_to_call(
            task_id=function_args.get("task_id"),
            assigned_to=function_args.get("assigned_to"),
            time_estimate=function_args.get("time_estimate"),
            due_date=function_args.get("due_date"),
            status=function_args.get("status")
        )

    message = {
        "tool_call_id": tool_call.id,  #
        "role": "tool",
        "name": function_name,
        "content": function_response,  # send the function response to the model, it's the JSON string of the function response
    }
    return message


def run_conversation(user_prompt):

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt} 
    ]
    tools= [
        tool_task_retrieval_and_status_updates,
        tool_project_reporting_and_analytics,
        tool_resource_allocation_and_scheduling
    ]

    # Use openai.ChatCompletion.create for openai < 1.0
    # openai.chat.completions.create for openai > 1.0
    response = client.chat.completions.create(
        model="gpt-4o",  # "gpt-3.5-turbo-1106"
        messages=messages,
        tools=tools,
        tool_choice="auto",  # let the model decide which tool (function) to use
    )
    response_message = response.choices[0].message
    print(f"(first) response message: {response_message} \n")
    tool_calls = response_message.tool_calls # get the tool calls from the first response

    # end of first response, now we parse the response and call the functions the model identified from our tool list
    # check if the model wanted to call a function
    if tool_calls:
        messages.append(response_message)  # extend the conversation with the first response
        # send the info for each function call and function response to the model
        for tool_call in tool_calls: # iterate through the tool calls in the response
            message_to_append = execute_function(tool_call)
            messages.append(message_to_append)  # extend conversation with function response

        # See https://gist.github.com/gaborcselle/2dc076eae23bd219ff707b954c890cd7
        #messages[1].content = "" # clear the first message (parsing bug)
        messages[1]['content'] = "" # clear the first message (parsing bug)
        print(f"message (with output from tool): {messages} \n")

        response = client.chat.completions.create(
            model="gpt-4o",  # "gpt-3.5-turbo-1106"
            messages=messages,
        )  # get a new response from the model where it can see the function response
    
    return response

#### Function Test: task_retrieval_and_status_updates

In [7]:
user_prompt = "Change the status of task 1 to completed (today date 2024-01-01)"
reponse = run_conversation(user_prompt)
print(f"Response from LLM: \n{reponse} \n")

message = reponse.choices[0].message
print(f"Final Message Content: \n{message.content} \n")

display_and_delete_file("project_management.csv")

(first) response message: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wob8BHaIsBRICK7k5E6BjDCm', function=Function(arguments='{"task_id":1,"status":"completed","last_updated":"2024-01-01"}', name='task_retrieval_and_status_updates'), type='function')]) 

message (with output from tool): [{'role': 'system', 'content': "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. \nYou can provide updates on projects, assign tasks to team members, and schedule meetings. \nYou understand project management terminology and are capable of parsing detailed project data. \nDon't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': ''}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatC

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,Jane Doe,completed,High,2023-08-01,2023-07-01,2024-01-01,10,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,completed,Medium,2023-08-15,2023-07-01,2023-12-15,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


File 'project_management.csv' has been deleted.


#### Function Test: project_reporting_and_analytics

In [8]:
user_prompt = "Describe Project 101's progress"
reponse = run_conversation(user_prompt)
print(f"Response from LLM: \n{reponse} \n")

message = reponse.choices[0].message
print(f"Final Message Content: \n{message.content} \n")

display_and_delete_file("project_management.csv")

(first) response message: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VqPIHQLanDb1zM8FGvKpPnlT', function=Function(arguments='{"project_id":101}', name='project_reporting_and_analytics'), type='function')]) 

message (with output from tool): [{'role': 'system', 'content': "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. \nYou can provide updates on projects, assign tasks to team members, and schedule meetings. \nYou understand project management terminology and are capable of parsing detailed project data. \nDon't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': ''}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VqPIHQLanDb1

#### Function Test: resource_allocation_and_scheduling

In [9]:
user_prompt = "Update the task with ID 3 to assign it to Albert Einstein, set the time estimate to 20 hours, the due date to Valentine day (2024), and change its status to In Progress."
reponse = run_conversation(user_prompt)
print(f"Response from LLM: \n{reponse} \n")

message = reponse.choices[0].message
print(f"Final Message Content: \n{message.content} \n")

display_and_delete_file("project_management.csv")

(first) response message: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kKfbXKyYfk9XcOvUEOylqM30', function=Function(arguments='{"task_id":3,"assigned_to":"Albert Einstein","time_estimate":20,"due_date":"2024-02-14","status":"In Progress"}', name='resource_allocation_and_scheduling'), type='function')]) 

message (with output from tool): [{'role': 'system', 'content': "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. \nYou can provide updates on projects, assign tasks to team members, and schedule meetings. \nYou understand project management terminology and are capable of parsing detailed project data. \nDon't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': ''}, ChatCompletionMessage(content=None, refusal=None, role='assistant',

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,Jane Doe,In Progress,High,2023-08-01,2023-07-01,2023-07-10,10,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,completed,Medium,2023-08-15,2023-07-01,2023-12-15,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Albert Einstein,In Progress,Low,2024-02-14,2023-06-01,2023-07-05,20,2.0,Compile the weekly project status report,Reporting,NaN


File 'project_management.csv' has been deleted.


#### Function Test: cannot determine

In [10]:
user_prompt = "When is the earliest deadline?"
reponse = run_conversation(user_prompt)
print(f"Response from LLM: \n{reponse} \n")

message = reponse.choices[0].message
print(f"Final Message Content: \n{message.content} \n")

display_and_delete_file("project_management.csv")

(first) response message: ChatCompletionMessage(content='Could you please provide me with the specific project or tasks you need information about so I can identify the earliest deadline for you?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None) 

Response from LLM: 
ChatCompletion(id='chatcmpl-Aec3geOyOjp3K1KNF3nDO1bC86BWc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Could you please provide me with the specific project or tasks you need information about so I can identify the earliest deadline for you?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1734242780, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_9faba9f038', usage=CompletionUsage(completion_tokens=26, prompt_tokens=296, total_tokens=322, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_predictio